In [34]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [35]:
book_data = pd.read_csv('data/data.csv')
book_content = book_data['content'].tolist()

import re
import string
import jieba

# 加载停用词
with open("dict/stop_words.utf8", encoding="utf8") as f:
    stopword_list = f.readlines()

#切分词去除换行符 
def tokenize_text(text):
    tokens = jieba.lcut(text)
    tokens = [token.strip() for token in tokens]
    return tokens

#去除特殊符号
def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub('', token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

#去除停用词
def remove_stopwords(text):
    tokens = tokenize_text(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ''.join(filtered_tokens)
    return filtered_text

#标准化
def normalize_corpus(corpus):
    normalized_corpus = []
    for text in corpus:

        text =" ".join(jieba.lcut(text))
        normalized_corpus.append(text)

    return normalized_corpus
# normalize corpus
norm_book_content = normalize_corpus(book_content)

norm_book_content

['现代人 内心 流失 的 东西 ， 这家 杂货店 能 帮 你 找回 — — \r\n 僻静 的 街道 旁有 一家 杂货店 ， 只要 写下 烦恼 投进 卷帘门 的 投 信口 ， 第二天 就 会 在 店 后 的 牛奶箱 里 得到 回答 。 \r\n 因 男友 身患 绝 ...',
 '在 第一次世界大战 的 硝烟 中 ， 每 一个 迈向 死亡 的 生命 都 在 热烈 地 生长 — — 威尔士 的 矿工 少年 、 刚 失恋 的 美国 法律系 大学生 、 穷困潦倒 的 俄国 兄弟 、 富有 英俊 的 英格兰 伯爵 ， 以及 痴 ...',
 '一个 三十几岁 的 美丽 女人 子君 ， 在家 做 全职 家庭主妇 。 却 被 一个 平凡 女人 夺走 丈夫 ， 一段 婚姻 的 失败 ， 让 女主角 不得不 坚强 ， 变得 更 美丽 ， 有 了 事业 ， 最终 遇见 一个 更 值得 爱 的 男 ...',
 '《 百年孤独 》 是 魔幻现实主义 文学 的 代表作 ， 描写 了 布恩迪亚 家族 七代 人 的 传奇 故事 ， 以及 加勒比海 沿岸 小镇 马孔多 的 百年 兴衰 ， 反映 了 拉丁美洲 一个 世纪 以来 风云变幻 的 历史 。 ...',
 '12 岁 的 阿富汗 富家 少爷 阿米尔 与 仆人 哈桑 情同手足 。 然而 ， 在 一场 风筝 比赛 后 ， 发生 了 一件 悲惨 不堪 的 事 ， 阿米尔 为 自己 的 懦弱 感到 自责 和 痛苦 ， 逼 走 了 哈桑 ， 不久 ， 自己 也 跟 ...',
 '令人 心碎 卻 無能 為力 的 真實 故事 。 \r\n 性 、 權力 、 升學主義 ─ ─ 青澀 的 表皮 、 變態 社會 的 日常 \r\n 如果 這件 事情 正在 發生 ， 我們 要 怎麼 假裝 世界 上 沒 有人 以 強暴 小女孩 為樂 ？ \r\n 「 我下 ...',
 '只有 你 身为 女人   才 会 知道 这些 丑陋 的 秘密 \r\n 两个 女人   50 年 的 友谊 和 战争 \r\n 如何 成为 那个 更 强大 的 她 ， 又 不 被 她 战胜 \r\n 那不勒斯 四部曲 NO.2 \r\n 《 我 的 天才 女友 》 后续 ，   ...',
 '编辑 推荐 : \r\n 1995 福克纳 奖得主 \r\n 占据 《 纽约时报 》 畅销 榜

In [43]:
#获取tf-idf特征
V=TfidfVectorizer(ngram_range=(1,2),min_df=2,max_df=0.7,use_idf=True,smooth_idf=True)
M=V.fit_transform(norm_book_content)
feature_names = V.get_feature_names()

In [44]:
from sklearn.cluster import KMeans


def k_means(feature_matrix, num_clusters=10):
    km = KMeans(n_clusters=num_clusters,
                max_iter=1000)
    km.fit(feature_matrix)
    clusters = km.labels_
    return km, clusters


num_clusters = 10
km_obj, clusters = k_means(feature_matrix=M,
                           num_clusters=num_clusters)
book_data['Cluster'] = clusters


In [40]:
def get_cluster_data(clustering_obj, book_data,
                     feature_names, num_clusters,
                     topn_features=10):
    cluster_details = {}
    # 获取cluster的center
    ordered_centroids = clustering_obj.cluster_centers_.argsort()[:, ::-1]
    # 获取每个cluster的关键特征
    # 获取每个cluster的书
    for cluster_num in range(num_clusters):
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster_num'] = cluster_num
        key_features = [feature_names[index]
                        for index
                        in ordered_centroids[cluster_num, :topn_features]]
        cluster_details[cluster_num]['key_features'] = key_features

        books = book_data[book_data['Cluster'] == cluster_num]['title'].values.tolist()
        cluster_details[cluster_num]['book_nums'] = len(books)
        cluster_details[cluster_num]['books'] = books

    return cluster_details

In [41]:
def print_cluster_data(cluster_data):
    # print cluster details
    for cluster_num, cluster_details in cluster_data.items():
        print('Cluster {} details:'.format(cluster_num))
        print('-' * 20)
        print('Key features:', cluster_details['key_features'])
        print('book_nums:',cluster_details['book_nums'])
        print('book in this cluster:')
        print(', '.join(cluster_details['books']))
        print('=' * 40)

In [45]:
cluster_data = get_cluster_data(clustering_obj=km_obj,
                                book_data=book_data,
                                feature_names=feature_names,
                                num_clusters=num_clusters,
                                topn_features=5)

print_cluster_data(cluster_data)

Cluster 0 details:
--------------------
Key features: ['哈利', '哈利 波特', '波特', '开始', '魔法']
book_nums: 32
book in this cluster:
明朝那些事儿（1-9）, 放学后, 1995-2005夏至未至, 放学后, 放学后, 明朝那些事儿（壹）, 明朝那些事儿（1-9）, ﻿1995-2005夏至未至, 夏至未至, 1995-2005夏至未至, 哈利·波特与魔法石, 哈利·波特与阿兹卡班的囚徒, 哈利·波特与火焰杯, 哈利·波特与死亡圣器, 哈利·波特与密室, 哈利·波特与混血王子, 哈利·波特与凤凰社, ﻿哈利·波特与魔法石, 哈利·波特与被诅咒的孩子, 哈利·波特与阿兹卡班的囚徒, 哈利·波特与火焰杯, 哈利·波特与死亡圣器, 哈利·波特与密室, 哈利·波特与混血王子, 哈利·波特与凤凰社, 诗翁彼豆故事集, 哈利•波特(共6册) (精装), 哈利·波特百科全书, 哈利·波特百科全书, 明朝那些事儿（1-9）, 明朝那些事儿（壹）, 明朝那些事儿（壹）
Cluster 1 details:
--------------------
Key features: ['故事', '爱情', '一个', '漫画', '作品']
book_nums: 205
book in this cluster:
我的前半生, 霍乱时期的爱情, 戴上手套擦泪, 霍乱时期的爱情, 戴上手套擦泪, 灯塔, 霍乱时期的爱情, 目送, 撒哈拉的故事, 活着本来单纯, 白夜行, 目送, 活着本来单纯, 撒哈拉的故事, 我口袋里的星辰如沙砾, 此生多珍重, 诗的八堂课, 夏洛的网, 天蓝色的彼岸, 安吉拉·卡特的精怪故事集, 王小波全集, 万寿寺, 王小波全集, 窗边的小豆豆, 我亲爱的甜橙树, 夏洛的网, 草房子, 天蓝色的彼岸, 基督山伯爵, 傲慢与偏见, 小团圆, 钱钟书散文, 基督山伯爵, 傲慢与偏见, 昨日之旅, 茨威格小说集, 杜拉斯谈杜拉斯, 抵挡太平洋的堤坝, 树犹如此, 孤獨，或類似的東西, 深夜食堂 01, 画的秘密, 方向, 睡魔1：前奏与夜曲, 我可以咬一口吗, 波丽娜, ﻿白夜行, 玫瑰的名字, S., ﻿你今天真好看, 我可以咬一口吗, 画的秘密, 向左走·向右